In [5]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from neo4j import GraphDatabase

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

USERNAME = os.getenv('MONGO_USERNAME')
PASSWORD = os.getenv('MONGO_PASSWORD')
URL = os.getenv('MONGO_URL')

NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
URI = os.getenv('NEO4J_URI')

In [7]:
username = USERNAME
password = PASSWORD
cluster_url = URL

uri = f"mongodb+srv://{username}:{password}@{cluster_url}/?retryWrites=true&w=majority"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

mongo_db = client["basketball_database"]

# Retrieve collections
players = mongo_db["players"].find()
teams = mongo_db["teams"].find()
relationships = mongo_db["relationships"].find()

In [8]:
# Neo4j Aura connection details
uri = URI
username = "neo4j"
password = NEO4J_PASSWORD
AUTH = (username, password)

driver = GraphDatabase.driver(uri, auth=(username, password))

# Create a session
session = driver.session()

In [61]:
for player in players:
    session.run(
        """
        MERGE (p:Player {name: $name, age: $age, number: $number, height: $height, weight: $weight})
        """,
        name=player["name"],
        age=player["age"],
        number=player["number"],
        height=player["height"],
        weight=player["weight"]
    )

# Create Team nodes
for team in teams:
    session.run(
        """
        MERGE (t:Team {name: $name})
        """,
        name=team["name"]
    )

# Create Relationships
for rel in relationships:
    if rel["type"] == "PLAYS_FOR":
        session.run(
            """
            MATCH (p:Player {name: $player_name}), (t:Team {name: $team_name})
            MERGE (p)-[:PLAYS_FOR {salary: $salary}]->(t)
            """,
            player_name=rel["player"],
            team_name=rel["team"],
            salary=rel["salary"]
        )
    elif rel["type"] == "TEAMMATES":
        session.run(
            """
            MATCH (p1:Player {name: $player1_name}), (p2:Player {name: $player2_name})
            MERGE (p1)-[:TEAMMATES]->(p2)
            MERGE (p2)-[:TEAMMATES]->(p1)
            """,
            player1_name=rel["player1"],
            player2_name=rel["player2"]
        )
    elif rel["type"] == "COACHES":
        session.run(
            """
            MATCH (p:Player {name: $player_name}), (c:Coach {name: $coach_name})
            MERGE (c)-[:COACHES]->(p)
            """,
            player_name=rel["player"],
            coach_name=rel["coach"]
        )
    elif rel["type"] == "PLAYED_AGAINST":
        session.run(
            """
            MATCH (p:Player {name: $player_name}), (t:Team {name: $team})
            MERGE (p)-[:PLAYED_AGAINST {team: $team, minutes: $minutes, points: $points, assists: $assists, rebounds: $rebounds, turnovers: $turnovers}]->(t)
            """,
            player_name=rel["player"],
            team=rel["team"],
            minutes=rel["minutes"],
            points=rel["points"],
            assists=rel["assists"],
            rebounds=rel["rebounds"],
            turnovers=rel["turnovers"]
        )

# Close the Neo4j session and driver
session.close()
driver.close()

print("Data has been successfully inserted into Neo4j!")

Data has been successfully inserted into Neo4j!
